[View in Colaboratory](https://colab.research.google.com/github/RahulVallivel/ipython_tf/blob/master/TensorFlow_with_GPU.ipynb)

# Confirm TensorFlow can see the GPU

Simply select "GPU" in the Accelerator drop-down in Notebook Settings (either through the Edit menu or the command palette at cmd/ctrl-shift-P).

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [7]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

KeyboardInterrupt: ignored

# Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# pylint: disable=unused-import


import numpy

import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
mnist = read_data_sets("./tmp/data", one_hot=True) # The data will be stored in ./tmp/data on your machine.
num_train = 55000 # Total number of training examples

batch_size = 50 # The size of a minibatch
epochs = 100 # Train on the entire training set for this much time
lr = 0.01 # Learning rate
Xte = mnist.test.images # Test data
Yte = mnist.test.labels # Test labels

# Explicitly set variables in the gpu memory. If you don't have a GPU, comment the 'with tf.device('/gpu"0')' line,
# and remove the following indents.
with tf.device('/gpu:0'):
    x = tf.placeholder(tf.float32, [None, 784], name='x') # A placeholder to put our input data
    y = tf.placeholder(tf.float32, [None, 10], name='y') # A placeholder to put our input labels
    W1 = tf.Variable(tf.random_normal([784, 100],mean=0, stddev=10.0), dtype=tf.float32, name='weights') # Model weights (trainable)
    W2 = tf.Variable(tf.random_normal([100,10],mean=0, stddev=10.0), dtype=tf.float32, name='weights')
    #b1 = tf.Variable(tf.random_uniform([1, 10]), dtype=tf.float32, name='bias') # Model bias (trainable)
    pred = tf.nn.sigmoid(tf.matmul(x, W1))
    pred2= tf.nn.softmax(tf.matmul(pred,W2))
    pred1=tf.matmul(pred,W2)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred1), name='loss') # Model loss function
    optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss) # Use basic gradient descent optimizer
    test_prediction = tf.equal(tf.argmax(pred1, 1), tf.argmax(y, 1)) # Test whether the predictions match real labels
    accuracy = tf.reduce_mean(tf.cast(test_prediction, tf.float32))

# Using minibatch to train the model.
num_batch = num_train / batch_size
init = tf.global_variables_initializer()
# Train the model
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        cost_this_epoch = 0
        for i in range(int(num_batch)):
            xtr, ytr = mnist.train.next_batch(batch_size)
            _, l = sess.run([optimizer, loss], feed_dict={x: xtr, y: ytr})
            cost_this_epoch += l * batch_size
        print('Epoch {} done. Loss: {:5f}'.format(epoch, cost_this_epoch))
        if(epoch%10==0):
          accr = sess.run(accuracy, feed_dict={x: Xte, y: Yte})
          print('Accuracy is {}%'.format(accr * 100))

NameError: ignored